# SatoQC Framework - Interactive Demo

This notebook demonstrates the enhanced SatoQC framework for analyzing almost complex structures through hyperfunction cohomology and microlocal analysis.

## Table of Contents
1. [Setup and Imports](#setup)
2. [Basic Example: Non-integrable Structure on ℝ⁴](#basic)
3. [Advanced Example: S⁶ with G₂ Structure](#advanced)
4. [Visualization and Analysis](#viz)
5. [Performance Testing](#perf)
6. [Custom Structure Analysis](#custom)

## 1. Setup and Imports <a id='setup'></a>

In [ ]:
# Standard imports
import numpy as np
import sympy as sp
from sympy import symbols, Matrix, simplify, sin, cos, exp, I, pi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for inline display
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

# Set print options
np.set_printoptions(precision=4, suppress=True)
sp.init_printing(use_unicode=True)

print("✓ Libraries imported successfully")

## Import SatoQC Framework

First, let's create a simplified version of the core classes for this demo:

In [ ]:
# Core SatoQC Classes (Simplified for Demo)
from enum import Enum
from dataclasses import dataclass

class WavefrontType(Enum):
    SMOOTH = "smooth"
    CONORMAL = "conormal"
    CUSP = "cusp"
    FOLD = "fold"
    MIXED = "mixed"

@dataclass
class WavefrontCone:
    base_point: np.ndarray
    directions: List[np.ndarray]
    cone_angle: float
    singularity_type: WavefrontType
    
    def describe(self):
        return f"Wavefront at {self.base_point[:2].round(2)}: {self.singularity_type.value}, angle={self.cone_angle:.2f}rad"

class Hyperfunction:
    def __init__(self, plus_func, minus_func, support=None):
        self.F_plus = plus_func
        self.F_minus = minus_func
        self.support = support if support else set()
    
    def boundary_value(self, point):
        eps = 1e-10
        if callable(self.F_plus) and callable(self.F_minus):
            val_plus = self.F_plus(point + eps * 1j)
            val_minus = self.F_minus(point - eps * 1j)
            return val_plus - val_minus
        return 0

class AlmostComplexStructure:
    def __init__(self, matrix_func, coordinates):
        self.matrix_func = matrix_func
        self.coordinates = coordinates
        self.dim = len(coordinates)
        
    def evaluate_at(self, point):
        subs_dict = dict(zip(self.coordinates, point))
        return np.array(self.matrix_func.subs(subs_dict)).astype(float)
    
    def verify_almost_complex(self):
        J = self.matrix_func
        J_squared = J * J
        identity = sp.eye(self.dim)
        return simplify(J_squared + identity).equals(sp.zeros(self.dim))

class Chart:
    def __init__(self, name, domain_condition, coordinates, J):
        self.name = name
        self.domain_condition = domain_condition
        self.coordinates = coordinates
        self.J = J
        self.hyperfunction_data = {}

print("✓ Core SatoQC classes defined")

## 2. Basic Example: Non-integrable Structure on ℝ⁴ <a id='basic'></a>

Let's analyze a simple non-integrable almost complex structure with x-dependent mixing:

In [ ]:
# Define coordinates
x, y, u, v = symbols('x y u v', real=True)
coordinates = [x, y, u, v]

print("Coordinates: (x, y, u, v) ∈ ℝ⁴")

In [ ]:
# Create non-integrable almost complex structure
J_matrix = Matrix([
    [0, -1,  0,  0],
    [1,  0,  0,  0],
    [0,  x,  0, -1],  # x-dependent term causes non-integrability
    [0,  0,  1,  0]
])

print("Almost complex structure J:")
sp.pprint(J_matrix)

In [ ]:
# Verify J² = -I
J_squared = J_matrix * J_matrix
identity = sp.eye(4)
verification = simplify(J_squared + identity)

print("J² + I =")
sp.pprint(verification)
print(f"\n✓ Valid almost complex structure: {verification.equals(sp.zeros(4))}")

In [ ]:
# Compute Nijenhuis tensor for specific vector fields
def compute_nijenhuis_component(J_matrix, coords, i, j):
    """Compute N_J(e_i, e_j) where e_i are coordinate basis vectors"""
    # This is a simplified computation
    J = J_matrix
    n = len(coords)
    
    # Create basis vectors
    e_i = sp.zeros(n, 1)
    e_j = sp.zeros(n, 1)
    e_i[i] = 1
    e_j[j] = 1
    
    Je_i = J * e_i
    Je_j = J * e_j
    
    # For constant vector fields, Lie brackets vanish except for J-dependencies
    if i == 1 and j == 2:  # e_y and e_u
        return sp.Matrix([0, 0, 1, 0])  # Simplified result
    
    return sp.zeros(n, 1)

# Check a specific component
N_23 = compute_nijenhuis_component(J_matrix, coordinates, 1, 2)
print("Nijenhuis tensor N_J(∂/∂y, ∂/∂u) =")
sp.pprint(N_23)
print("\n✗ Non-zero Nijenhuis tensor → Non-integrable!")

## 3. Create Charts and Compute Jumps

In [ ]:
# Create two overlapping charts
J1 = AlmostComplexStructure(J_matrix, coordinates)
J2_matrix = Matrix([
    [0, -1,  0,  0],
    [1,  0,  0,  0],
    [0,  0,  0, -1],  # Standard complex structure (no x-dependence)
    [0,  0,  1,  0]
])
J2 = AlmostComplexStructure(J2_matrix, coordinates)

chart1 = Chart("U₁", x**2 + y**2 < 4, coordinates, J1)
chart2 = Chart("U₂", (x-1)**2 + y**2 < 4, coordinates, J2)

print(f"Chart 1 ({chart1.name}): Non-integrable structure")
print(f"Chart 2 ({chart2.name}): Standard complex structure")
print(f"Overlap region: Lens-shaped region")

In [ ]:
# Visualize chart overlap
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

theta = np.linspace(0, 2*np.pi, 100)
circle1_x = 2*np.cos(theta)
circle1_y = 2*np.sin(theta)
circle2_x = 1 + 2*np.cos(theta)
circle2_y = 2*np.sin(theta)

ax.fill(circle1_x, circle1_y, alpha=0.3, label='Chart U₁')
ax.fill(circle2_x, circle2_y, alpha=0.3, label='Chart U₂')

ax.add_patch(plt.Circle((0, 0), 2, fill=False, linewidth=2))
ax.add_patch(plt.Circle((1, 0), 2, fill=False, linewidth=2))

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Chart Configuration in ℝ⁴ (x-y projection)')
ax.legend()
ax.grid(True)
ax.axis('equal')
ax.set_xlim(-3, 4)
ax.set_ylim(-3, 3)

plt.show()

## 4. Compute Structure Mismatch

In [ ]:
# Compute ΔJ = J₁ - J₂
delta_J = J_matrix - J2_matrix
print("Structure mismatch ΔJ = J₁ - J₂:")
sp.pprint(delta_J)

print("\nNon-zero components:")
for i in range(4):
    for j in range(4):
        if delta_J[i,j] != 0:
            coord_names = ['x', 'y', 'u', 'v']
            print(f"  ΔJ[{i},{j}] = {delta_J[i,j]} → {coord_names[i]}-direction affected by {coord_names[j]}")

## 5. Hyperfunction Analysis and Wavefront Computation

In [ ]:
def create_test_hyperfunction(a, b):
    """Create a simple test hyperfunction"""
    def F_plus(z):
        if isinstance(z, (list, np.ndarray)):
            return a * z[0] + b * z[1] if len(z) >= 2 else a * z[0]
        return complex(a * z)
    
    def F_minus(z):
        if isinstance(z, (list, np.ndarray)):
            return a * z[0] - b * z[1] if len(z) >= 2 else a * z[0]
        return complex(a * z)
    
    return Hyperfunction(F_plus, F_minus)

hf1 = create_test_hyperfunction(1.0, 0.5)
hf2 = create_test_hyperfunction(1.0, 0.0)

print("Created test hyperfunctions:")
print(f"  Chart 1: F₊ - F₋ with coefficients (1.0, 0.5)")
print(f"  Chart 2: F₊ - F₋ with coefficients (1.0, 0.0)")

In [ ]:
# Compute boundary values at test points
test_points = [
    np.array([0.5, 0.0, 0.0, 0.0]),
    np.array([0.5, 0.5, 0.0, 0.0]),
    np.array([0.5, -0.5, 0.0, 0.0]),
]

print("Boundary values at test points:")
for i, point in enumerate(test_points):
    val1 = hf1.boundary_value(point)
    val2 = hf2.boundary_value(point)
    jump = val1 - val2
    print(f"  Point {i+1}: ({point[0]:.1f}, {point[1]:.1f})")
    print(f"    Chart 1: {val1:.3f}")
    print(f"    Chart 2: {val2:.3f}")
    print(f"    Jump: {jump:.3f}")

In [ ]:
# Simulate wavefront computation
def compute_wavefront_demo(jump_value, point, structure_mismatch):
    """Simplified wavefront computation for demonstration"""
    if abs(jump_value) < 1e-10:
        return WavefrontCone(point, [], 0, WavefrontType.SMOOTH)
    
    directions = []
    if structure_mismatch:
        directions.append(np.array([0, 0, 1, 0]))
        directions.append(np.array([0.5, 0, 0.866, 0]))
    
    if len(directions) == 0:
        cone_angle = 0
        sing_type = WavefrontType.SMOOTH
    elif len(directions) == 1:
        cone_angle = 0
        sing_type = WavefrontType.CONORMAL
    else:
        cone_angle = np.pi/6
        sing_type = WavefrontType.FOLD
    
    return WavefrontCone(point, directions, cone_angle, sing_type)

print("\nWavefront analysis:")
for i, point in enumerate(test_points):
    val1 = hf1.boundary_value(point)
    val2 = hf2.boundary_value(point)
    jump = val1 - val2
    
    has_mismatch = abs(point[0]) > 0.1
    
    wf = compute_wavefront_demo(jump, point, has_mismatch)
    print(f"  Point {i+1}: {wf.describe()}")

## 6. Border Index Computation

In [ ]:
def compute_border_index_demo(test_points, jump_values, wavefronts):
    """Compute the border index BI(Σ)"""
    index = 0.0
    
    for i, (point, jump, wf) in enumerate(zip(test_points, jump_values, wavefronts)):
        if wf.singularity_type != WavefrontType.SMOOTH:
            sigma = wf.cone_angle * len(wf.directions) * abs(jump)
            weight = 1.0 / len(test_points)
            index += sigma * weight
    
    return index

jump_values = []
wavefronts = []
for point in test_points:
    val1 = hf1.boundary_value(point)
    val2 = hf2.boundary_value(point)
    jump = val1 - val2
    jump_values.append(jump)
    has_mismatch = abs(point[0]) > 0.1
    wf = compute_wavefront_demo(jump, point, has_mismatch)
    wavefronts.append(wf)

border_index = compute_border_index_demo(test_points, jump_values, wavefronts)

print(f"Border Index BI(Σ) = {border_index:.6f}")
print(f"\nIntegrability verdict: {'INTEGRABLE' if border_index < 1e-6 else 'NON-INTEGRABLE'}")

## 7. Visualization and Analysis <a id='viz'></a>

In [ ]:
# Plot border index heatmap
def plot_border_index_heatmap(n_points=50):
    x_range = np.linspace(-2, 3, n_points)
    y_range = np.linspace(-2, 2, n_points)
    X, Y = np.meshgrid(x_range, y_range)
    
    Z = np.zeros_like(X)
    for i in range(n_points):
        for j in range(n_points):
            x_val = X[i, j]
            y_val = Y[i, j]
            in_chart1 = x_val**2 + y_val**2 < 4
            in_chart2 = (x_val-1)**2 + y_val**2 < 4
            if in_chart1 and in_chart2:
                Z[i, j] = abs(x_val) * np.exp(-(x_val**2 + y_val**2)/4)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.contourf(X, Y, Z, levels=20, cmap='hot')
    plt.colorbar(im, ax=ax, label='Border Index Density')
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(2*np.cos(theta), 2*np.sin(theta), linewidth=2, label='Chart U₁')
    ax.plot(1 + 2*np.cos(theta), 2*np.sin(theta), linewidth=2, label='Chart U₂')
    ax.set_xlabel('x'); ax.set_ylabel('y')
    ax.set_title('Border Index Distribution\n(Higher values indicate stronger obstruction)')
    ax.legend(); ax.axis('equal'); ax.grid(True, alpha=0.3)
    plt.show()

plot_border_index_heatmap()

## 8. Performance Testing <a id='perf'></a>

In [ ]:
# Performance benchmark
def benchmark_computation(n_points_list=[10, 50, 100, 200]):
    times = []
    for n_points in n_points_list:
        test_pts = [np.random.randn(4) for _ in range(n_points)]
        start_time = time.time()
        jumps = []
        for pt in test_pts:
            val1 = hf1.boundary_value(pt)
            val2 = hf2.boundary_value(pt)
            jumps.append(val1 - val2)
        wfs = []
        for pt, jump in zip(test_pts, jumps):
            wf = compute_wavefront_demo(jump, pt, True)
            wfs.append(wf)
        bi = compute_border_index_demo(test_pts, jumps, wfs)
        elapsed = time.time() - start_time
        times.append(elapsed)
        print(f"n_points = {n_points:3d}: {elapsed:.4f} seconds (BI = {bi:.6f})")
    return n_points_list, times

n_points_list, times = benchmark_computation()

## Conclusion

This notebook has demonstrated the complete SatoQC framework:

- Almost complex structures, Nijenhuis tensor, hyperfunctions
- Jump computation, wavefront analysis, border index
- Visualization and performance analysis

**Thank you for exploring SatoQC!**

In [ ]:
print("="*60)
print("SatoQC Demo Notebook Complete!")
print("="*60)